### imports
***

In [1]:
%run preprocess.ipynb

/usr/local/lib/python3.7/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/home/iis519/.local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


recall bm25...



0.0501626583013826
recall idf...



0.0072905774369699085
recall s2v...



0.023178807947019868
recall blue...


...

(2204890, 8)
cpu: 48
used cpu: 16


Process ForkPoolWorker-4:
Process ForkPoolWorker-12:
Process ForkPoolWorker-7:
Process ForkPoolWorker-11:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-9:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Process ForkPoolWorker-8:
Process ForkPoolWorker-15:
Process ForkPoolWorker-16:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/

  File "<ipython-input-1-3f2566e9308a>", line 324, in get_features
    data[prefix + 'key_in_psu_key_number_da_11'] = list(map(lambda x,y: get_num_psu_key(x,y,11,False),data['description_text_pre'],data['abstract_pre']))
  File "<ipython-input-1-3f2566e9308a>", line 327, in get_features
    data[prefix + 'key_in_psu_key_number_rate_ka_12'] = list(map(lambda x,y: get_num_psu_key(x,y,12),data['key_text_pre'],data['abstract_pre']))
  File "<ipython-input-1-3f2566e9308a>", line 317, in <lambda>
    data[prefix + 'key_in_psu_key_number_rate_ka_11'] = list(map(lambda x,y: get_num_psu_key(x,y,11),data['key_text_pre'],data['abstract_pre']))
  File "<ipython-input-1-3f2566e9308a>", line 323, in get_features
    data[prefix + 'key_in_psu_key_number_dt_11'] = list(map(lambda x,y: get_num_psu_key(x,y,11,False),data['description_text_pre'],data['title_pro']))
  File "<ipython-input-1-3f2566e9308a>", line 334, in get_features
    data[prefix + 'key_in_psu_key_number_da_12'] = list(map(lambda x,y: ge

  File "<ipython-input-1-3f2566e9308a>", line 263, in filter_len
    ret = [word for word in string.split() if len(word) >= MIN_LEN and word in word2idf]
  File "<ipython-input-1-3f2566e9308a>", line 268, in top_k
    ret = sorted(string.split(), key=lambda x: len(x), reverse=True)[:k]
  File "<ipython-input-1-3f2566e9308a>", line 263, in <listcomp>
    ret = [word for word in string.split() if len(word) >= MIN_LEN and word in word2idf]
KeyboardInterrupt
KeyboardInterrupt
  File "<ipython-input-1-3f2566e9308a>", line 263, in <listcomp>
    ret = [word for word in string.split() if len(word) >= MIN_LEN and word in word2idf]
  File "<ipython-input-1-3f2566e9308a>", line 268, in <lambda>
    ret = sorted(string.split(), key=lambda x: len(x), reverse=True)[:k]
  File "<ipython-input-1-3f2566e9308a>", line 263, in <listcomp>
    ret = [word for word in string.split() if len(word) >= MIN_LEN and word in word2idf]
  File "<ipython-input-1-3f2566e9308a>", line 263, in <listcomp>
    ret = [wor

KeyboardInterrupt: 

In [2]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import multiprocessing as mp
import pandas as pd
import numpy as np
import re
import time
from tqdm.notebook import tqdm
from util import pre_process
import pickle
import gc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [3]:
data_path = '../../data_2020/'
model_path = 'models/'
other_path = 'others/'
n = 80
drop_no_content = True
drop_no_year = True
drop_keywords = False
drop_no_abstract = False
MIN_LEN = 7
workers = mp.cpu_count()//2
paper_thd = 45000
num_leaves = 64
reg_alpha = 1
reg_lambda = 0.1
objective = 'binary'
max_depth = -1
learning_rate = 0.1
min_child_samples = 5
n_estimators = 5000
subsample = 0.8
colsample_bytree = 0.8
seed = 9487

### load data
***

In [4]:
# for prediction
test = pd.read_csv(data_path+'validation.csv')

# get meta
train_data = pd.read_csv(data_path+'train_data_merge_{}_2.csv'.format(n))[['description_id','label','journal']]
test_data = pd.read_csv(data_path+'test_data_merge_{}_{}_2.csv'.format(n, paper_thd))[['paper_id','description_id','journal','year','keywords']].fillna('')
abstract = pd.read_csv(data_path+'test_data_merge_{}_{}_2.csv'.format(n, paper_thd))[['abstract']].fillna('NO_CONTENT')
print(train_data.shape, test_data.shape)

# drop
keep_idxs = []
if drop_no_content:
    test_have_content_idxs = test_data.index[test_data['journal'] != 'no-content'].tolist()
    keep_idxs += test_have_content_idxs
    print(len(keep_idxs))
if drop_no_year:
    test_have_year_idxs = test_data.index[test_data['year'] != ''].tolist()
    if len(keep_idxs) > 0:
        keep_idxs = list(set(test_have_year_idxs).intersection(set(keep_idxs)))
    else:
        keep_idxs += test_have_year_idxs
    print(len(keep_idxs))
if drop_keywords:
    test_no_keywords_idxs = test_data.index[test_data['keywords'] == ''].tolist()
    if len(keep_idxs) > 0:
        keep_idxs = list(set(test_no_keywords_idxs).intersection(set(keep_idxs)))
    else:
        keep_idxs += test_have_year_idxs
    print(len(keep_idxs))
if drop_no_abstract:
    test_have_abstract_idxs = abstract.index[abstract['abstract'] != 'NO_CONTENT'].tolist()
    if len(keep_idxs) > 0:
        keep_idxs = list(set(test_have_abstract_idxs).intersection(set(keep_idxs)))
    else:
        keep_idxs += test_have_abstract_idxs
    print(len(keep_idxs))
        
# sort indexs
keep_idxs = sorted(keep_idxs)
if not drop_no_content and not drop_no_year and not drop_keywords and not drop_no_abstract:
    keep_idxs = [i for i in range(test_data.shape[0])]
    
# get feature
train_x = pd.read_csv(data_path+'train_data_merge_{}_featall_3.csv'.format(n))
test_x = pd.read_csv(data_path+'test_data_merge_{}_{}_featall_2.csv'.format(n, paper_thd))
train_y = train_data[['label']]
print(train_x.shape, test_x.shape, train_y.shape)
test_x = test_x.iloc[keep_idxs]
test_data = test_data.iloc[keep_idxs]

test_x.shape, test_data.shape

FileNotFoundError: [Errno 2] File b'../../data_2020/train_data_merge_80_2.csv' does not exist: b'../../data_2020/train_data_merge_80_2.csv'

In [ ]:
# # drop feature
# drop = 'psu_key'
# cols = [col for col in test_x.columns.values if not col.__contains__(drop)]
# train_x = train_x[cols]
# test_x = test_x[cols]
# train_x.shape, test_x.shape

### train model - lgb
***

In [ ]:
def lgb_train(train_x,train_y,valid_x,valid_y):
    print('train_x shape:',train_x.shape)
    lgb_model = lgb.LGBMClassifier(num_leaves=num_leaves,
                                   reg_alpha=reg_alpha,
                                   reg_lambda=reg_lambda,
                                   objective=objective,
                                   max_depth=max_depth,
                                   learning_rate=learning_rate,
                                   min_child_samples=min_child_samples,
                                   n_estimators=n_estimators,
                                   subsample=subsample,
                                   colsample_bytree=colsample_bytree,
                                   n_jobs=workers,
                                   random_state=seed)
    lgb_model.fit(train_x, train_y,
                  eval_set=[(valid_x, valid_y)],
                  eval_metric='auc',
                  early_stopping_rounds=100,
                  verbose=100)
    # im
    f_importance = pd.DataFrame(columns=['f_name', 'importance'])
    f_importance['importance'] = lgb_model.feature_importances_
    f_importance['f_name'] = train_x.columns.values
    f_importance['f_score'] = f_importance['importance'] / f_importance['importance'].sum()
    f_importance.sort_values(['f_score'], ascending=False, inplace=True)
    f_importance.drop(columns=['importance'], axis=1, inplace=True)
    print(f_importance)
    f_importance.to_csv('im_2.csv',index=False)
    return lgb_model

### CV
***

In [ ]:
%%time
n_splits = 10
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
train_preds = np.zeros(train_x.shape[0])
test_preds = []
models = []

for fold_, (train_idx, val_idx) in enumerate(folds.split(train_x, train_y)):
    print("fold n°{}".format(fold_+1))
    train_flod_x = train_x.iloc[train_idx]
    train_flod_y = train_y.iloc[train_idx]
    validate_flod_x = train_x.iloc[val_idx]
    validate_flod_y = train_y.iloc[val_idx]
    model = lgb_train(train_flod_x,train_flod_y,validate_flod_x,validate_flod_y)
    train_preds[val_idx] = model.predict_proba(validate_flod_x)[:, 1]
    test_preds.append(model.predict_proba(test_x)[:, 1])
    models.append(model)

### save & load model
***

In [ ]:
# save
for i in range(len(models)):
    with open(model_path+'lgb_new_{}_2'.format(i), 'wb') as f:
        pickle.dump(models[i], f)

In [ ]:
# # load
# models = []
# for i in range(10):
#     with open(model_path+'lgb_{}'.format(i), 'rb') as f:
#         models.append(pickle.load(f))

### prediction
***

In [ ]:
%%time
# vote
def get_topn_paper_id(paper_id_list,preb_list,tag):
    index = np.argsort(preb_list)
    if len(index) < tag:
        tag = len(index)
        return np.nan
    return paper_id_list[index[-tag]]

did2vote = {did[0]: {} for did in test[['description_id']].fillna('none').values}

for test_pred in tqdm(test_preds):
    test_data['pred'] = test_pred
    temp = test_data[['description_id','pred']].groupby('description_id')['pred'].agg(lambda x: list(x))
    test_data['preb_list'] = test_data['description_id'].map(temp)
    temp = test_data[['description_id','paper_id']].groupby('description_id')['paper_id'].agg(lambda x: list(x))
    test_data['paper_id_list'] = test_data['description_id'].map(temp)
    test_sub = test_data.drop_duplicates('description_id')

    test_sub['p1'] = list(map(lambda x,y:get_topn_paper_id(x,y,1),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p2'] = list(map(lambda x,y:get_topn_paper_id(x,y,2),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p3'] = list(map(lambda x,y:get_topn_paper_id(x,y,3),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p4'] = list(map(lambda x,y:get_topn_paper_id(x,y,4),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p5'] = list(map(lambda x,y:get_topn_paper_id(x,y,5),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p6'] = list(map(lambda x,y:get_topn_paper_id(x,y,6),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p7'] = list(map(lambda x,y:get_topn_paper_id(x,y,7),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p8'] = list(map(lambda x,y:get_topn_paper_id(x,y,8),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub = test_sub[['description_id','p1','p2','p3','p4','p5','p6','p7','p8']]
    
    # vote
    weights = {1: 10, 2: 8, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1}
    for row in test_sub.values:
        for i in range(1, 9):
            if row[i] not in did2vote[row[0]]:
                did2vote[row[0]][row[i]] = weights[i]
            else:
                did2vote[row[0]][row[i]] += weights[i]

In [ ]:
# get top3
test_sub = []
for did in tqdm(list(did2vote.keys())):
    tmp = [(key, did2vote[did][key]) for key in list(did2vote[did].keys())]
    tmp.sort(key=lambda x: x[1], reverse=True)
    top3 = []
    for pid in tmp:
        if type(pid[0]) != float:
            top3.append(pid[0])
            if len(top3) >= 3:
                break
    test_sub.append([did]+top3)
test_sub = pd.DataFrame(test_sub, columns=['description_id','p1','p2','p3'])

sub = test[['description_id']]
sub = sub.merge(test_sub, on='description_id', how='left')
sub.to_csv('predictions/prediction_{}_{}_small.csv'.format(seed, paper_thd), index=False, sep=',', header=None)

In [ ]:
sum(train_preds)/len(train_preds)

In [ ]:
sum(sum(test_preds)/len(test_preds[0]))/len(models)

In [ ]:
sub[pd.isnull(sub).any(axis=1)]